# Lesson 2.5: Practical Application - Building a Content Generation App

---

In previous lessons, we explored the core components of LangChain such as **Prompts**, **Models**, **Output Parsers**, and how to connect them into **Chains** using **LangChain Expression Language (LCEL)**. This lesson will be a comprehensive practical exercise where you will apply all that knowledge to build a simple but useful application: a content generation tool for blog posts.

## 1. Applying Learned Concepts

This content generation application will illustrate how you can chain multiple LLM tasks together to achieve a larger goal. Specifically, we will use:

* **`ChatPromptTemplate`**: To guide the LLM for each specific task.
* **`ChatOpenAI` (or `ChatGoogleGenerativeAI`)**: As the main language model.
* **`StrOutputParser`**: To extract string responses.
* **`CommaSeparatedListOutputParser`**: To receive a list of ideas.
* **LCEL (`|`, `RunnableParallel`)**: To efficiently connect sequential processing steps.


---

## 2. Designing the Content Generation Application

Our application will perform three main steps, each being a separate Chain, which are then connected sequentially:

1.  **Idea Generation:** Based on an input topic, the LLM will suggest a few blog post ideas.
2.  **Outline Development:** Choosing one idea from step 1, the LLM will generate a detailed outline for that blog post.
3.  **Introduction Writing:** Based on the generated outline, the LLM will write an engaging introductory paragraph for the blog post.




---

## 3. Building the Application Components

First, ensure you have installed the necessary libraries and set up your API keys as learned in Lesson 1.4.

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai pydantic

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Initialize Chat Model
# Use a lower temperature for more stable and less randomly creative output
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

### 3.1. Step 1: Generate Blog Post Ideas Based on a Topic

We will ask the LLM to provide a list of ideas.

* **Goal:** Get 3-5 blog post ideas on a specific topic.
* **Input:** `topic` (blog post topic).
* **Output:** A list of ideas.
* **Components:** `ChatPromptTemplate` -> `llm` -> `CommaSeparatedListOutputParser`.

In [ ]:
# --- Chain 1: Generate blog ideas ---
idea_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a content marketing expert. Suggest 5 engaging blog post ideas about the following topic, separated by commas."),
    HumanMessage(content="Topic: {topic}"),
])

idea_chain = idea_prompt | llm | CommaSeparatedListOutputParser()

# Test Chain 1 independently
# topic_test = "artificial intelligence in education"
# ideas_test = idea_chain.invoke({"topic": topic_test})
# print(f"Blog ideas for '{topic_test}': {ideas_test}")

### 3.2. Step 2: Develop a Detailed Outline for the Chosen Idea

After getting ideas, we will choose one and ask the LLM to develop an outline.

* **Goal:** Create a detailed outline (including sections, subheadings) for a blog post idea.
* **Input:** `blog_idea` (the chosen blog idea).
* **Output:** Free-form text outline.
* **Components:** `ChatPromptTemplate` -> `llm` -> `StrOutputParser`.

In [ ]:
# --- Chain 2: Develop detailed outline ---
outline_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a content editor. Create a detailed outline for the following blog post. The outline should include an introduction, main sections, subsections, and a conclusion."),
    HumanMessage(content="Blog post idea: {blog_idea}"),
])

outline_chain = outline_prompt | llm | StrOutputParser()

# Test Chain 2 independently
# idea_test = "How AI Personalizes Learning Experiences"
# outline_test = outline_chain.invoke({"blog_idea": idea_test})
# print(f"\nOutline for '{idea_test}':\n{outline_test}")

### 3.3. Step 3: Write an Introductory Paragraph for the Blog Post

Finally, we will use the outline to write the introductory paragraph.

* **Goal:** Write an engaging introductory paragraph based on the outline.
* **Input:** `blog_outline` (detailed outline).
* **Output:** Text introduction.
* **Components:** `ChatPromptTemplate` -> `llm` -> `StrOutputParser`.

In [ ]:
# --- Chain 3: Write blog post introduction ---
intro_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a professional writer. Based on the following outline, write an engaging and captivating introductory paragraph for the blog post. The introduction should introduce the topic and hook the reader."),
    HumanMessage(content="Blog post outline:\n{blog_outline}"),
])

intro_chain = intro_prompt | llm | StrOutputParser()

# Test Chain 3 independently
# outline_example = """
# I. Introduction
#    A. Opening on AI and education
#    B. Importance of personalized learning
# II. How AI personalizes learning?
#    A. Analyzing learning data
#    B. Content recommendation systems
#    C. Instant feedback
# III. Benefits of AI in personalization
#    A. Enhanced learning effectiveness
#    B. Reduced teacher workload
# IV. Challenges and future
#    A. Ethical and privacy issues
#    B. Future development
# V. Conclusion
# """
# intro_test = intro_chain.invoke({"blog_outline": outline_example})
# print(f"\nIntroduction:\n{intro_test}")


---

## 4. Connecting Steps into a Sequential Chain using LCEL

Now, we will connect these three small Chains into a larger sequential chain using LCEL. We will use `RunnableParallel` to manage inputs and pass results between steps.

In [ ]:
# --- Connect the entire application using LCEL ---

# Step 1: Idea Generation
# Input: {"topic": "..."}
# Output: list of ideas (e.g., ["idea1", "idea2", "idea3"])
# We will select the first idea from this list to proceed
full_content_chain = (
    {"ideas": idea_chain, "original_topic": RunnablePassthrough()} # Run idea_chain and retain original topic
    | RunnableParallel(
        # Get the first idea from the ideas list to use as input for outline_chain
        blog_idea=lambda x: x["ideas"][0],
        # Retain the original topic for later use if needed
        original_topic=lambda x: x["original_topic"]
    )
    | RunnableParallel(
        # Run outline_chain with blog_idea
        blog_outline=outline_chain,
        # Pass through blog_idea and original_topic
        blog_idea=lambda x: x["blog_idea"],
        original_topic=lambda x: x["original_topic"]
    )
    | RunnableParallel(
        # Run intro_chain with blog_outline
        blog_intro=intro_chain,
        # Pass through blog_outline, blog_idea, original_topic
        blog_outline=lambda x: x["blog_outline"],
        blog_idea=lambda x: x["blog_idea"],
        original_topic=lambda x: x["original_topic"]
    )
)

# Data flow notes:
# 1. The initial input for `full_content_chain.invoke()` is a dictionary, e.g., `{"topic": "..."}`.
# 2. The first step `{"ideas": idea_chain, "original_topic": RunnablePassthrough()}`
#    - `idea_chain` receives `{"topic": ...}` and generates `ideas` (list of strings).
#    - `RunnablePassthrough()` receives the entire original input and assigns it to `original_topic`.
#    - The output of this step is `{"ideas": ["...", "..."], "original_topic": {"topic": "..."}}`.
# 3. The second step `| RunnableParallel(blog_idea=lambda x: x["ideas"][0], ...)`
#    - Takes `ideas[0]` from the previous output and assigns it to `blog_idea`.
#    - Passes `original_topic` through.
#    - The output of this step is `{"blog_idea": "...", "original_topic": {"topic": "..."}}`.
# 4. The third step `| RunnableParallel(blog_outline=outline_chain, ...)`
#    - `outline_chain` receives `{"blog_idea": ...}` and generates `blog_outline`.
#    - Passes `blog_idea` and `original_topic` through.
#    - The output of this step is `{"blog_outline": "...", "blog_idea": "...", "original_topic": {"topic": "..."}}`.
# 5. The fourth step `| RunnableParallel(blog_intro=intro_chain, ...)`
#    - `intro_chain` receives `{"blog_outline": ...}` and generates `blog_intro`.
#    - Passes `blog_outline`, `blog_idea`, `original_topic` through.
#    - The final output is a dictionary containing `blog_intro`, `blog_outline`, `blog_idea`, `original_topic`.


---

## 5. Executing the Content Generation Application

Now we will run the entire chain with a specific topic and observe the results.

In [ ]:
# Execute the entire chain
blog_topic = "future of work and AI"

print(f"--- Generating blog content on topic: '{blog_topic}' ---")
full_response = full_content_chain.invoke({"topic": blog_topic})

print("\n--- Suggested blog ideas ---")
# Note: full_response['ideas'] would be the list of ideas from the first step
# However, in the chain, we only took ideas[0] to pass into blog_idea.
# To display all ideas, you would need to adjust RunnableParallel or run idea_chain separately.
# Here, we'll assume the first idea was chosen.
print(f"Selected idea: {full_response['blog_idea']}")

print("\n--- Detailed Outline ---")
print(full_response["blog_outline"])

print("\n--- Blog Post Introduction ---")
print(full_response["blog_intro"])

print("\n--- Completed ---")

You can try changing the `blog_topic` and re-running the program to see different content generated.


---

## Lesson Summary

This lesson provided a comprehensive practical exercise on building a simple content generation application using LangChain. We applied our knowledge of **Prompts**, **Models**, **Output Parsers**, and **LangChain Expression Language (LCEL)** to create a sequential processing chain consisting of three steps: **generating blog post ideas**, **developing a detailed outline**, and **writing an introductory paragraph**. By using the `|` operator and `RunnableParallel`, we effectively connected these smaller Chains, illustrating how data flows through the steps and how you can build more complex workflows with LangChain. This practical exercise is a crucial step for you to start designing and deploying your own LLM applications.